### playnomore
- http://playnomore.co.kr/
- scrapy에서 fake-useragent 사용
- scrapy를 실행할때 아규먼트를 설정해서 실행
- pipelines에서 데이터 베이스로 데이터를 저장

In [5]:
import scrapy
import requests
from scrapy.http import TextResponse

#### 1. 프로젝트 생성

In [6]:
!rm -rf playnomore
!scrapy startproject playnomore

rm: cannot remove 'playnomore': Device or resource busy


New Scrapy project 'playnomore', using template directory 'C:\ProgramData\Anaconda3\lib\site-packages\scrapy\templates\project', created in:
    c:\Users\jlune\carbon\01.MachineLearning\91.크롤링\playnomore

You can start your first spider with:
    cd playnomore
    scrapy genspider example example.com


#### 2. items.py
- title, price, img, link

In [7]:
%%writefile playnomore/playnomore/items.py
import scrapy

class PlaynomoreItem(scrapy.Item):
    title = scrapy.Field()
    price = scrapy.Field()
    img = scrapy.Field()
    link = scrapy.Field()

Overwriting playnomore/playnomore/items.py


#### 3. xpath 확인
- 링크
- 링크 -> 상세페이지(제목, 이미지URL, 가격)
- fake_useragent 설치
    - pip install fake_useragent

In [8]:
from fake_useragent import UserAgent
url = "http://playnomore.co.kr/category/bag/24/"
# headers = { "User-Agent": UserAgent().chrome }
headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" }
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8") 
response

<200 http://playnomore.co.kr/category/bag/24/>

In [20]:
# 링크
links = response.xpath(
    '//*[@id="contents"]/div[2]/div/ul/li/div/a/@href'
).extract()
links = list(map(response.urljoin, links))
# urljoin이란 기능은 전체 URl "http://playnomore.co.kr/category/bag/24/" 에 + 세부 url을 붙여주는 기능을 한다.

In [17]:
# links = response.xpath(
#     '//*[@id="contents"]/div[2]/div/ul/li/div/a/@href'
# ).extract()
# links[0]

'/product/detail.html?product_no=572&cate_no=24&display_group=1'

In [16]:
# links

['http://playnomore.co.kr/product/detail.html?product_no=572&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=574&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=507&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=506&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=580&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=581&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=552&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=534&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=532&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=575&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=573&cate_no=24&display_group=1',
 'http://p

In [21]:
links

['http://playnomore.co.kr/product/detail.html?product_no=572&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=574&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=507&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=506&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=580&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=581&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=552&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=534&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=532&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=575&cate_no=24&display_group=1',
 'http://playnomore.co.kr/product/detail.html?product_no=573&cate_no=24&display_group=1',
 'http://p

In [7]:
# 상세페이지 : 제목, 가격, 이미지URL
url = links[0]
headers = { "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36" }
req = requests.get(url, headers=headers)
response = TextResponse(req.url, body=req.text, encoding="utf-8") 
response

<200 http://playnomore.co.kr/product/20off-180-micro-baguette-white/572/?cate_no=24&display_group=1>

In [8]:
title1 = response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()'
    ).extract()
    # //*[@id="contents"]/div[1]/div[1]/div[2]/div[1]
    # //*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()[1]
title2 = response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()'
    ).extract()
title = "".join(title1) + "".join(title2)
price = response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()'
    ).extract()[0]
img = "http:" + response.xpath(
        '//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src'
    ).extract()[0]
title, price, img, title1, title2

('20%off  MICRO BAGUETTE white',
 '$ 144',
 'http://playnomore.co.kr/web/product/big/202007/ad23b70dcdd2091e4b243dd198c6f414.jpg',
 [],
 ['20%off ', ' MICRO BAGUETTE ', 'white'])

#### 4. spider.py
- scrapy-fake-useragent 설치
    - pip install scrapy-fake-useragent

In [24]:
!pip list | grep fake

fake-useragent                     0.1.11
scrapy-fake-useragent              1.4.4


In [23]:
%%writefile playnomore/playnomore/spiders/spider.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = "Playnomore"
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
        }
    }
    
    def start_requests(self):
        url = "http://playnomore.co.kr/category/bag/24/"
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse)
    
    def page_parse(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()').extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item["title"] = "".join(title1) + "".join(title2)
        item["price"] = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()').extract()[0]
        item["img"] = "http:" + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src').extract()[0]
        item["link"] = response.url
        yield item

Overwriting playnomore/playnomore/spiders/spider.py


In [11]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore.csv

Overwriting run.sh


In [12]:
!chmod +x run.sh

In [13]:
!./run.sh

'.'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [14]:
import pandas as pd
df = pd.read_csv("playnomore/playnomore.csv")
df.tail(1)

FileNotFoundError: [Errno 2] No such file or directory: 'playnomore/playnomore.csv'

#### 5. argument 설정

In [ ]:
%%writefile playnomore/playnomore/spiders/spider.py
import scrapy
from playnomore.items import PlaynomoreItem

class PlaynomoreSpider(scrapy.Spider):
    name = "Playnomore"
    custom_settings = {
        'DOWNLOADER_MIDDLEWARES': {
            'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware': None,
            'scrapy_fake_useragent.middleware.RandomUserAgentMiddleware': 400,
        }
    }
    
    def __init__(self, category1="bag", category2=24, **kwargs):
        self.start_url = "http://playnomore.co.kr/category/{}/{}/".format(category1, category2)
        super().__init__(**kwargs)
        
    def start_requests(self):
        url = self.start_url
        yield scrapy.Request(url, callback=self.parse)
        
    def parse(self, response):
        links = response.xpath('//*[@id="contents"]/div[2]/div/ul/li/div/a/@href').extract()
        links = list(map(response.urljoin, links))
        for link in links:
            yield scrapy.Request(link, callback=self.page_parse)
    
    def page_parse(self, response):
        item = PlaynomoreItem()
        title1 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/font/text()').extract()
        title2 = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[1]/text()').extract()
        item["title"] = "".join(title1) + "".join(title2)
        item["price"] = response.xpath('//*[@id="contents"]/div[1]/div[1]/div[2]/div[2]/text()').extract()[0]
        item["img"] = "http:" + response.xpath('//*[@id="contents"]/div[1]/div[1]/div[1]/div[1]/img/@src').extract()[0]
        item["link"] = response.url
        yield item

Overwriting playnomore/playnomore/spiders/spider.py


In [ ]:
%%writefile run.sh
cd playnomore
scrapy crawl Playnomore -o playnomore2.csv -a category1=apparel -a category2=26

Overwriting run.sh


In [ ]:
!./run.sh

'.'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [ ]:
import pandas as pd
df = pd.read_csv("playnomore/playnomore2.csv")
df.tail(1)

,img,link,price,title
14,http://playnomore.co.kr/web/product/big/202007...,http://playnomore.co.kr/product/20off-180-micr...,$ 144,20%off MICRO BAGUETTE white


In [ ]:
df["img"]

0     http://playnomore.co.kr/web/product/big/202101...
1     http://playnomore.co.kr/web/product/big/201910...
2     http://playnomore.co.kr/web/product/big/202007...
3     http://playnomore.co.kr/web/product/big/202009...
4     http://playnomore.co.kr/web/product/big/202103...
5     http://playnomore.co.kr/web/product/big/201910...
6     http://playnomore.co.kr/web/product/big/201910...
7     http://playnomore.co.kr/web/product/big/202007...
8     http://playnomore.co.kr/web/product/big/202007...
9     http://playnomore.co.kr/web/product/big/202101...
10    http://playnomore.co.kr/web/product/big/202101...
11    http://playnomore.co.kr/web/product/big/202007...
12    http://playnomore.co.kr/web/product/big/202102...
13    http://playnomore.co.kr/web/product/big/202007...
14    http://playnomore.co.kr/web/product/big/202007...
Name: img, dtype: object

#### 6. Mongodb에 저장
- pymongo를 pipelines.py에 적용
- pip install pymongo==2.8.1

In [ ]:
!pip list | grep pymongo

pymongo               2.8.1    


In [ ]:
import pymongo

In [ ]:
# client = pymongo.MongoClient('mongodb://13.125.153.195:27017/')
# client
# 내거 3.35.111.73으로 설정
client = pymongo.MongoClient('mongodb://3.35.111.73:27017/')
client

MongoClient(host=['3.35.111.73:27017'], document_class=dict, tz_aware=False, connect=True)

In [ ]:
db = client.playnomore
collection = db.shoes
collection

Collection(Database(MongoClient(host=['3.35.111.73:27017'], document_class=dict, tz_aware=False, connect=True), 'playnomore'), 'shoes')

In [ ]:
data = {"title":"신발"}
collection.insert(data)

TypeError: 'Collection' object is not callable. If you meant to call the 'insert' method on a 'Collection' object it is failing because no such method exists.

##### Mongodb 모듈 파일 생성

In [ ]:
%%writefile playnomore/playnomore/mongodb.py
import pymongo

client = pymongo.MongoClient('mongodb://13.125.153.195:27017/')
db = client.playnomore
collection = db.shoes

Writing playnomore/playnomore/mongodb.py


In [ ]:
%%writefile playnomore/playnomore/pipelines.py
from .mongodb import collection

class PlaynomorePipeline(object):
    
    def process_item(self, item, spider):
        
        data = { "title": item["title"], 
                 "price": item["price"],
                 "img": item["img"], 
                 "link": item["link"],
               }
        
        collection.insert(data)
        
        return item

Overwriting playnomore/playnomore/pipelines.py


In [ ]:
!echo "ITEM_PIPELINES = {"  >> playnomore/playnomore/settings.py

In [ ]:
!echo "   'playnomore.pipelines.PlaynomorePipeline': 300," >> playnomore/playnomore/settings.py

In [ ]:
!echo "}" >> playnomore/playnomore/settings.py

In [ ]:
!tail -n 5 playnomore/playnomore/settings.py

#HTTPCACHE_IGNORE_HTTP_CODES = []
#HTTPCACHE_STORAGE = 'scrapy.extensions.httpcache.FilesystemCacheStorage'
ITEM_PIPELINES = {
    playnomore.pipelines.PlaynomorePipeline: 300,
}


In [ ]:
!cat run.sh

cd playnomore
scrapy crawl Playnomore -o playnomore2.csv -a category1=shoes -a category2=25


In [ ]:
!./run.sh

Traceback (most recent call last):
  File "/home/ubuntu/.pyenv/versions/python3/bin/scrapy", line 8, in <module>
    sys.exit(execute())
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/cmdline.py", line 114, in execute
    settings = get_project_settings()
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/utils/project.py", line 69, in get_project_settings
    settings.setmodule(settings_module_path, priority='project')
  File "/home/ubuntu/.pyenv/versions/3.6.9/envs/python3/lib/python3.6/site-packages/scrapy/settings/__init__.py", line 294, in setmodule
    module = import_module(module)
  File "/home/ubuntu/.pyenv/versions/3.6.9/lib/python3.6/importlib/__init__.py", line 126, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 994, in _gcd_import
  File "<frozen importlib._bootstrap>", line 971, in _find_and_load
  File "<frozen